In [1]:
from imap_tools import MailBox, AND
import os
import pandas as pd
import re

In [ ]:
# username = "adm.happyenergy@gmail.com"
# password = "mxtl agjv dcsn tcnv" #adm.happyenergy@gmail.com
# _from = "engenhariahappysolar@gmail.com"

username = "engenhariahappysolar@gmail.com"
password = "jtpg hdvr pmat rtap" #engenhariahappysolar@gmail.com
_from = 'noreplyportalgd@neoenergia.com'

query = "Portal da Geração Distribuída: Solicitação "



: 

In [ ]:

mailbox = MailBox('imap.gmail.com').login(username, password)
lista_emails = mailbox.fetch(AND(from_=_from))
lista_emails_com_query = [email for email in lista_emails if query.lower() in email.subject.lower()]
print("Emails encontrados:", len(lista_emails_com_query))

In [ ]:
Clientes = []
for email in lista_emails_com_query:

    match_num = re.search(r'Solicitação (\d+)', email.subject)
    numero_solicitacao = match_num.group(1) if match_num else "Não encontrado"

    match_paragrafo = re.search(r'<p>Prezado\(a\) (.*?),', email.html, re.IGNORECASE | re.DOTALL)
    Nome_cliente = match_paragrafo.group(1).strip() if match_paragrafo else "Não encontrado"
    if Nome_cliente not in [c['nome_cliente'] for c in Clientes]:
        Clientes.append({
        'numero_solicitacao': numero_solicitacao,
        'nome_cliente': Nome_cliente,
        'data': email.date
    })
    # print(Nome_cliente)
    # break
len(Clientes)
df = pd.DataFrame(Clientes)
df.to_csv('clientes_all.csv', index=False)
print("Lista gerada e salva em clientes_all.csv")

Lista gerada e salva em clientes_all.csv


In [ ]:
BASE_DIR = os.getcwd()
# CSV_FILES = [
#     os.path.join(BASE_DIR, 'Solcitações/PortalGD_minhas_solicitacoes.csv'),
#     os.path.join(BASE_DIR, 'Solcitações/PortalGD_minhas_solicitacoes (1).csv'),
#]

CSV_FILES = [os.path.join(BASE_DIR, file) for file in os.listdir(BASE_DIR) if file.endswith('.csv')]

def load_active_codes():
    codes = set()
    for path in CSV_FILES:
        if os.path.exists(path):
            try:
                df = pd.read_csv(path, encoding='latin1')
            except Exception:
                df = pd.read_csv(path, encoding='utf-8', errors='ignore')
            if df.shape[1] >= 2:
                vals = df.iloc[:, 1].astype(str).str.strip()
                codes.update(vals.tolist())
    return codes


In [5]:
BASE_DIR = os.getcwd()

def get_all_clients(Directory):
    cont = 0
    CSV_FILES = [os.path.join(Directory, file) for file in os.listdir(Directory) if file.endswith('.csv')]
    for csv_file in CSV_FILES:
        df_temp = pd.read_csv(csv_file, encoding='latin1')
        if cont == 0:
            df = df_temp
            cont += 1
        else:
            df = pd.concat([df, df_temp], ignore_index=True)
    return df
# print(df.shape())

In [10]:
solicitacoes = get_all_clients(os.path.join(BASE_DIR, 'Solicitações'))
all_clients  = pd.read_csv('clientes_all.csv')

# Merge para adicionar a coluna nome
# solicitacoes = solicitacoes.merge(
#     all_clients[['Protocolo da Solicitação', 'nome_cliente']], 
#     on='numero_solicitacao', 
#     how='left'
# )

# # Renomear coluna se necessário
# solicitacoes = solicitacoes.rename(columns={'nome_cliente': 'nome'})

print(solicitacoes.head())
print(all_clients.head())


   Protocolo da Solicitação  Código da Unidade Consumidora  \
0                2601199327                      204364443   
1                2512232215                     7096579211   
2                2512055922                     7087990170   
3                2510242711                     7086746810   
4                2510087236                     7095208031   

                                   Status Atual Data de Ingresso  
0  Aguardando solicitação de vistoria e Conexão       19/01/2026  
1  Aguardando solicitação de vistoria e Conexão       29/12/2025  
2                         Solicitação Concluída       05/12/2025  
3                         Solicitação Concluída       24/10/2025  
4                         Solicitação Concluída       08/10/2025  
   numero_solicitacao                                 nome_cliente  \
0          2409116427                       NIVALDO ALVES FERREIRA   
1          2408061389                       FAUSTINO LUIZ DE SOUZA   
2          2408

In [12]:
df_final = pd.merge(
    solicitacoes, 
    all_clients, 
    left_on='Protocolo da Solicitação', 
    right_on='numero_solicitacao', 
    how='inner'
)

print(df_final.head())
df_final.to_csv('lista completa.csv', index=False)

   Protocolo da Solicitação  Código da Unidade Consumidora  \
0                2601199327                      204364443   
1                2512232215                     7096579211   
2                2512055922                     7087990170   
3                2510087236                     7095208031   
4                2510066351                     7096699640   

                                   Status Atual Data de Ingresso  \
0  Aguardando solicitação de vistoria e Conexão       19/01/2026   
1  Aguardando solicitação de vistoria e Conexão       29/12/2025   
2                         Solicitação Concluída       05/12/2025   
3                         Solicitação Concluída       08/10/2025   
4                         Solicitação Concluída       23/12/2025   

   numero_solicitacao                 nome_cliente                       data  
0          2601199327  SUELY ROMAO DANTAS DA SILVA  2026-01-19 17:27:38-03:00  
1          2512232215    Eudarc Goncalves de Sousa  2025-1

In [ ]:
mailbox = MailBox('imap.gmail.com').login(username, password)
lista_emails = mailbox.fetch(AND(from_=_from))